In [3]:
### Extract Text from HTML

from bs4 import BeautifulSoup
import os

#Generate markdowns
input_dir ="./data/project-assurance-data"
txt_dir = './data/txt_files'
os.makedirs(txt_dir,exist_ok=True)

for filename in os.listdir(input_dir):
    with open(input_dir+"/"+filename, "rb") as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    markdown = soup.get_text()
        
    new_filename = filename[:-4]+"txt"
    with open(txt_dir+"/"+new_filename, "w", encoding="utf-8") as file:
        file.write(markdown)

In [3]:
#### Chunk using semantic chunking
from chunking import semantic_chunking
import os
chunk_size = 500
overlap = 100

txt_dir = './data/txt_files'
chunk_dir = './data/chunks'
os.makedirs(chunk_dir,exist_ok=True)
for filename in os.listdir(txt_dir):
    #read file
    with open(txt_dir+"/"+filename, "r", encoding="utf-8") as file:
        text = file.read()

    #create chunk
    chunks = semantic_chunking(text, chunk_size, overlap)

    #save chunk
    for i,chunk in enumerate(chunks):
        new_filename = filename[:-4]+"_"+str(i)+".txt"

        with open(chunk_dir+"/"+new_filename, "w", encoding="utf-8") as file:
            text = file.write(chunk)

In [4]:
### Create Vectorstore
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from embeddings import E5Embeddings

# Initialize embeddings
embeddings = E5Embeddings()

# Construct documents
docs = []
for chunk_file in os.listdir(chunk_dir):
    with open(os.path.join(chunk_dir, chunk_file), "r", encoding="utf-8") as f:
        chunk = f.read()
    docs.append(Document(chunk))

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save
store_dir = "./vectorstore/"
os.makedirs(store_dir, exist_ok=True)
vectorstore.save_local(store_dir+"faiss_index")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: intfloat/e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
